# MNIST handwritten digits classification with support vector machines 

In this notebook, we'll use [support vector machines (SVMs)](http://scikit-learn.org/stable/modules/svm.html#svm-classification) and related algorithms to classify MNIST digits using scikit-learn.

First, the needed imports. 

In [ ]:
%matplotlib inline

from time import time
import numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

Then we load the MNIST data. First time it downloads the data, which can take a while.

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print()
print('MNIST data loaded: train:',len(X_train),'test:',len(X_test))
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

A support vector machine (SVM) is a discriminative classifier defined by a separating *hyperplane*. The training data is used to learn an optimal hyperplane that separates the training samples.  During inference, the hyperplane is used to classify the test samples.

![title](imgs/Svm_max_sep_hyperplane_with_margin_small.png)

<br/>

<center><small>Image by <a href="//commons.wikimedia.org/w/index.php?title=User:Cyc&amp;action=edit&amp;redlink=1" class="new" title="User:Cyc (page does not exist)">Cyc</a> - <span class="int-own-work" lang="en">Own work</span>, Public Domain, <a href="https://commons.wikimedia.org/w/index.php?curid=3566688">Link</a></small></center>

## Not linearly separable

To handle cases in which the data are not linearly separable, SVM uses a *hinge* loss function.

In [ ]:
x = np.arange(-3,3,.01)
plt.figure()
plt.plot(x, np.maximum(1-x,0), label='SVM: hinge')
plt.plot(x, np.square(np.maximum(1-x,0)), label='SVM: squared hinge')
plt.plot(x, np.log(1+np.exp(-x)), label='logistic regression')
plt.axis([-3, 3, -0.5, 4])
plt.title('Loss functions')
l = plt.legend()

## SVM demo

Let's start by playing with a nice [Javascript SVM demo](http://cs.stanford.edu/people/karpathy/svmjs/demo/).

## Multiclass classification

SVMs are by nature binary classifiers, so we have to do a bit of extra work to support multiclass classification. There are two general strategies to convert a K-way multiclass problem into binary classification:

- **one-vs-rest (ovr); one-vs-all (ova):** training *K* classifiers, that is, a single classifier per class, with the samples of that class as positive samples and all other samples as negatives.

- **one-vs-one (ovo):** training *K(K-1)/2* binary classifiers for all pairs of classes to learn to distinguish between these two classes.  At prediction time, a voting scheme is applied: all classifiers are applied to the test sample and the class that gets the highest number of predictions is the prediction result of the multiclass classifier.


## Linear SVM 

### Learning

Our first classifier is a linear SVM trained with a subset of training data.  Let's use the [`LinearSVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC) class, as it is a specialized in linear SVMs. `C` is the penalty parameter.  (The general `SVC` has a similar `kernel=’linear’` option that can also be used.  The third option is to use `SGDClassifier`.)

In [ ]:
t0 = time()
C = 1.0
clf_lsvm = svm.LinearSVC(C=C)
print(clf_lsvm.fit(X_train[:10000,:,:].reshape(-1,28*28), y_train[:10000]))
print('Time elapsed: %.2fs' % (time()-t0))

The training of a Linear SVM is rather fast, so it seems more data could easily be used in the training.

Note also that the default multiclass strategy of `LinearSVM` is one-vs-rest.

### Inference

As the decision boundaries are linear, prediction with linear SVMs is fast:

In [ ]:
pred_lsvm = clf_lsvm.predict(X_test.reshape(-1,28*28))
print('Predicted', len(pred_lsvm), 'digits with accuracy:', accuracy_score(y_test, pred_lsvm))

## Logistic regression

Logistic regression is closely related to linear SVM.  Instead of hinge loss, it uses a different loss function derived from a probabilistic model.  The corresponding scikit-learn class is [LogisticRegression](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression).

### Learning

In [ ]:
t0 = time()
clf_lr = LogisticRegression(solver="sag")
print(clf_lr.fit(X_train[:10000,:,:].reshape(-1,28*28), y_train[:10000]))
print('Time elapsed: %.2fs' % (time()-t0))

### Inference

In [ ]:
pred_lr = clf_lr.predict(X_test.reshape(-1,28*28))
print('Predicted', len(pred_lr), 'digits with accuracy:', accuracy_score(y_test, pred_lr))

## Stochastic gradient descent

The scikit-learn class for stochastic gradient descent classifiers is [`SDGClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier).

### Learning

Stochastic gradient descent can be used to train the model one sample at a time.  Depending on the `loss` parameter, it can be used for linear SVMs (`loss="hinge"` or `loss="squared_hinge"`) and logistic regression (`loss="log"`).


In [ ]:
t0 = time()
clf_sgd = SGDClassifier(loss="hinge")
print(clf_sgd.fit(X_train.reshape(-1,28*28), y_train))
print('Time elapsed: %.2fs' % (time()-t0))

### Inference

In [ ]:
pred_sgd = clf_sgd.predict(X_test.reshape(-1,28*28))
print('Predicted', len(pred_sgd), 'digits with accuracy:', accuracy_score(y_test, pred_sgd))

## Kernel SVM

In addition to linear classification, SVMs can be used for non-linear classification by implicitly mapping the input features into high-dimensional feature spaces.  This is sometimes called the *kernel trick*, as the implicit mapping is often computationally cheaper than explicitly operating in the high-dimensional space.

### Learning

Let's train a *3rd degree polynomial kernel SVM* with the *one-vs-rest* strategy for multiclass classification. A Gaussian kernel, that is `kernel='rbf'` is another common choice.

In [ ]:
t0 = time()
clf_ksvm = svm.SVC(decision_function_shape='ovr', kernel='poly', degree=3)
print(clf_ksvm.fit(X_train[:10000,:,:].reshape(-1,28*28), y_train[:10000]))
print('Time elapsed: %.2fs' % (time()-t0))

### Inference

Despite the kernel trick, prediction of new samples is noticeably slower than with the linear SVM.  The classification accuracy, on the other hand, is improved. 

In [ ]:
t0 = time()
pred_ksvm = clf_ksvm.predict(X_test.reshape(-1,28*28))
print('Predicted', len(pred_ksvm), 'digits with accuracy:', accuracy_score(y_test, pred_ksvm))
print('Time elapsed: %.2fs' % (time()-t0))

## Model tuning

Study the scikit-learn documentation of the linear and kernel [SVMs](http://scikit-learn.org/stable/modules/svm.html#svm) and the available SVM classes ([`SVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC), [`NuSVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVC.html#sklearn.svm.NuSVC), [`LinearSVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC), and [`SDGClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)). See also [`LogisticRegression`](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression). Experiment with different hyperparameter values.

Report the highest classification accuracy you manage to obtain. Also mark down the parameters you used, so others can try to reproduce your results.
